In [1]:
# import numpy and pandas, and DataFrame / Series
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
# Set some pandas options
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
# And some items for matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import logprocessing as lp
import io
from IPython.display import display, HTML

#http://songhuiming.github.io/pages/2017/04/02/jupyter-and-pandas-display/

In [2]:
m =  lp.MetadataBuilder().load_cpu_cores(32).build()

query = {
        "header": "timestamp cpu1 io_write io_read net_down net_up memory power",
        "columns": "cpu0.timestamp, 100 - cpu0.idle, io.writet, io.readt, net.download, net.upload, memory.used, mean( energy.power_active )"
}

def dataframe_from_exp( d ):
    buffer = io.StringIO()

    p = lp.LogProcessor( 
        directory = d,
        meta = m,
        query = query,
        _file = buffer)

    p.process()
    buffer.flush()
    buffer.seek(0)
    df = pd.read_csv( buffer, delimiter=' ', index_col = False)
    buffer.close()
    
    return df
    

In [3]:
df = dataframe_from_exp("../experiments/")

df

,timestamp,cpu1,io_write,io_read,net_down,net_up,memory,power


In [4]:
#simple monovariable linear regression model

import statsmodels.formula.api as smf

mod = smf.ols(formula='power ~ cpu1', data=df)
res = mod.fit()

print(res.summary())

ValueError: negative dimensions are not allowed